In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [3]:
target_size = 30*
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")
model_max_length = 4096

In [4]:
with open("input/student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [5]:
df = pd.read_csv("input/geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [7]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 4min 8s, sys: 3min 38s, total: 7min 47s
Wall time: 13min 46s


In [8]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [9]:
prompter = Prompter(target_size=target_size, student_names=student_names, street_addresses=street_addresses)
res = prompter.prompt()
print(res.prompt)

Result(prompt='[INST] Write a short essay on the following topic:\n        In Education Policy and Reform, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?\n        Intersperse the following information in different parts of the essay:\n        1. Give an example that occurred near Holt Road (The Horizon), at a school frequented by Arjun Menon\n        2. Give an example citing someone named ArjunMenon\n        3. Give an example citing: periscope.tv/officialarjuniammenon [/INST]\n        ', street_address='Holt Road (The Horizon)', student_name='Arjun Menon', username='ArjunMenon', personal_url='periscope.tv/officialarjuniammenon')


In [10]:
%%time
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [16:05<00:00, 96.51s/it]

CPU times: user 15min 14s, sys: 51.3 s, total: 16min 5s
Wall time: 16min 5s


In [11]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            10 non-null     object
 1   prompt          10 non-null     object
 2   street_address  10 non-null     object
 3   student_name    10 non-null     object
 4   username        10 non-null     object
 5   personal_url    10 non-null     object
 6   model           10 non-null     object
dtypes: object(7)
memory usage: 688.0+ bytes


In [12]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In the field of architecture and urban design, visualization tools have become increasingly important for fostering innovation and collaboration in a design thinking approach. These tools allow architects and designers to communicate their ideas more effectively, leading to better outcomes for all stakeholders involved in a project.\n\n One example of this can be seen near Jalan Teck Whye, where Block 1","[INST] Write a short essay on the following topic:\n In Architecture and Urban Design, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Jalan Teck Whye (Block 154), at a nature park frequented by Aarti Mehta\n 2. Give an example citing someone named _4807.aarti-iam--mehta\n 3. Give an example citing: periscope.tv/aartimehtaxxx [/INST]\n",Jalan Teck Whye (Block 154),Aarti Mehta,_4807.aarti-iam--mehta,periscope.tv/aartimehtaxxx,Mixtral-8x7B-Instruct-v0.1
1,"Sustainable urban development is a crucial aspect of modern city planning, as it aims to create cities that are environmentally friendly, socially inclusive, and economically viable. One approach to fostering innovation and collaboration in this field is through the use of storytelling within the design thinking process. This method allows for a more empathetic understanding of the users' needs and experiences, which can ultimately lead to the creation of more sustainable and equitable urban spaces.\n\nAn example of the power of storytelling in sustainable urban development can be seen near Pasir Ris Drive 10, specifically at the subway station frequented by Nurul Aisyah. Nurul is a young woman who relies heavily on public transportation to get to work, school, and other daily activities. However, she often faces challenges in navigating the station due to its poor lighting, confusing layout, and lack of accessibility features. Through the use of storytelling, designers and urban planners were able to understand Nurul's experiences and incorporate her feedback into the redesign of the station. By doing so, they were able to create a space that better meets the needs of users like Nurul, making it more accessible, safe, and comfortable.\n\nAnother example of storytelling being used in sustainable urban development can be seen through the work of __iamNurul_xxx.Aisyah. As an active blogger and social media influencer, she uses her platform to share her personal experiences living in a dense, urban environment. Through her stories, she highlights both the successes and failures of urban planning and design, encouraging her followers to think critically about their own cities and the ways in which they can be improved. For instance, she has written extensively about the need for more green spaces in her community, sparking conversations and collaborations around this issue.\n\nA final example can be found on the blogspot.me/@iamnurulaisyah. The blogger shares her experience of using a bike-sharing system in her city and provides insights on how it could be improved to better serve the community, particularly for those with mobility issues. Her stories offer valuable firsthand knowledge and perspectives, highlighting areas where the bike-sharing system falls short in terms of sustainability, inclusivity, and accessibility. These insights can then be used to inform the redesign and implementation of the bike-sharing system, resulting in a more user-cent","[INST] Write a short essay on the following topic:\n In Sustainable Urban Development, how can storytelling be used within the design thinking process to foster innovation and collaboration?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Pasir Ris Drive 10 (Block 701), at a subway station frequented by Nurul Aisyah\n 2. Give an example ci

In [13]:
df.to_parquet("output/part01.parquet")
assert df.notna().all(axis=None)

In [14]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:29:52.182096
